In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from pyimagesearch.lenet import LeNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

Using TensorFlow backend.


In [2]:
# initialize the number of epochs to train for, initia learning rate,
# and batch size
EPOCHS = 1
INIT_LR = 1e-3
BS = 18

In [3]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [4]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("train")))
random.seed(42)
random.shuffle(imagePaths)

In [5]:
# loop over the input images
for imagePath in imagePaths:
        # load the image, pre-process it, and store it in the data list
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (480, 270))
        image = img_to_array(image)
        data.append(image)

        # extract the class label from the image path and update the
        # labels list
        label = imagePath.split(os.path.sep)[-2]
        label = 1 if label == "real" else 0
        labels.append(label)

In [6]:
data[0]

array([[[ 11.,  17.,  30.],
        [ 11.,  17.,  30.],
        [ 11.,  17.,  31.],
        ...,
        [ 43.,  50.,  65.],
        [ 43.,  50.,  65.],
        [ 43.,  50.,  65.]],

       [[ 11.,  17.,  30.],
        [ 11.,  17.,  30.],
        [ 12.,  18.,  31.],
        ...,
        [ 43.,  50.,  65.],
        [ 43.,  50.,  65.],
        [ 43.,  50.,  65.]],

       [[ 12.,  18.,  31.],
        [ 12.,  18.,  31.],
        [ 13.,  18.,  32.],
        ...,
        [ 43.,  50.,  65.],
        [ 43.,  50.,  65.],
        [ 43.,  50.,  65.]],

       ...,

       [[ 91., 122., 175.],
        [ 91., 122., 175.],
        [ 91., 122., 175.],
        ...,
        [ 68.,  84., 113.],
        [ 68.,  84., 113.],
        [ 68.,  84., 113.]],

       [[ 90., 120., 174.],
        [ 90., 120., 174.],
        [ 90., 121., 174.],
        ...,
        [ 68.,  85., 113.],
        [ 68.,  85., 113.],
        [ 68.,  85., 113.]],

       [[ 89., 120., 173.],
        [ 89., 120., 173.],
        [ 89., 1

In [7]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [8]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
        labels, test_size=0.25, random_state=42)

In [9]:
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [10]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True, fill_mode="nearest")

In [11]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=480, height=270, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
        metrics=["accuracy"])

[INFO] compiling model...


In [12]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
        validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
        epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/1
24/24 [==============================] - 189s 8s/step - loss: 5.9463 - accuracy: 0.5878 - val_loss: 5.5736 - val_accuracy: 0.6376


In [13]:
# save the model to disk
print("[INFO] serializing network...")
model.save("real_fake.model")

[INFO] serializing network...


In [14]:
H.history

{'val_loss': [5.573630924992913],
 'val_accuracy': [0.6375839114189148],
 'loss': [5.973264337981892],
 'accuracy': [0.587822]}

In [15]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on real/fake")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("model_evaluation.jpg")